# FormDiagram

In this first part of the tutorial, we will set up the FormDiagram.
We will create the geometry from a `meshgrid`, identify the supports, relax the unsupported boundaries, and, finally, update the boundary conditions needed for the TNA solver.

First, the relevant imports.

In [20]:
import compas
import pathlib
from compas_tna.diagrams import FormDiagram
from compas_tna.equilibrium import relax_boundary_openings

# for visualisation of the result
from compas.colors import Color
from compas_notebook.viewer import Viewer

## Create a FormDiagram from a meshgrid

Creating a diagram from a grid is super easy.
Just set the dimensions in x and y, and specify the number of grid cells in each directions.
If the grid is symmetrical, you only have to specify the properties in the x direction...

In [14]:
form = FormDiagram.from_meshgrid(dx=10, nx=10)

## Identify the supports

The corner vertices of the meshgrid are going to be the supports of the vault.
In a quadrilateral grid, like the meshgrid, the vertices in the corners have `degree == 2`.

In [15]:
corners = list(form.vertices_where(vertex_degree=2))
form.vertices_attribute("is_anchor", True, keys=corners)

## Unsupported boundaries

Next, we relax the unsupported boundaries in between the supports to give them an initial curvature that will allow them to resist the outward thrusting forces coming from the inner part of the vault, and guide them to the supports.

In [16]:
%%capture

form.edges_attribute("q", 10.0, keys=form.edges_on_boundary())
relax_boundary_openings(form, corners)

## Boundary conditions

Finally, we update the boundary conditions of the diagram to make it ready for TNA-based form finding.

In [17]:
form.update_boundaries()

## Export

Before visualising the result, we will export the FormDiagram in a form finding session, so it can be used as a starting point for the following steps in the process.

Note that there is no reliable way to identify the parent folder of the current notebook programmatically.
Therefore, the filepath below is hard-coded.
You should update it to whatever makes sense on your system.

In [21]:
session = {'formdiagram': form, 'forcediagram': None}
filepath = pathlib.Path('~/Code/compas_tna/notebooks/session.json').expanduser()

compas.json_dump(session, filepath)

## Visualisation

When updating the boundary conditions, some edges are marked as `_is_loaded=False` and some edges as `_is_edge=False`.
This is because these faces and edges are needed for the correct topological structure of the FormDiagram mesh, but have to be excluded from the equilibrium calculations. (Note that the lading underscore on both attribute names indicates that these attributes should not be modified by the user, in most use cases.)

This needs to be taken into account during visualisation...

In [18]:
viewer = Viewer()
viewer.scene.add(
    form,
    color=Color.from_hex('#cccccc'),
    show_faces=list(form.faces_where(_is_loaded=True)),
    show_edges=list(form.edges_where(_is_edge=True)),
    show_vertices=list(form.vertices_where(is_anchor=True))
)
viewer.show()